$\textbf{After removing the profession, city, state categorical columns}$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_predict
from sklearn.compose import ColumnTransformer

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score,classification_report,roc_auc_score


%matplotlib inline

!pip install xgboost
import xgboost as xgb
from xgboost import XGBClassifier


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Training Data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Test Data.csv')


In [ ]:
#test_data

In [ ]:

df = train_data.copy(True)
df2 = test_data.copy(True)

In [ ]:
df.drop(['Id', 'profession', 'city', 'state'], axis=1, inplace=True)
df2.drop(['id', 'profession', 'city', 'state'], axis=1, inplace=True)


In [ ]:
#df

In [ ]:
df['married'] = [0*i if df['married'][i]=='single' else 1 for i in range(len(df)) ]
df['house_ownership'] = [0 if df['house_ownership'][i]=='norent_noown' else 1 if df['house_ownership'][i]=='rented' else -1 for i in range(len(df))]
df['car_ownership'] = [0 if df['car_ownership'][i]=='no' else 1 for i in range(len(df))]

df2['married'] = [0*i if df2['married'][i]=='single' else 1 for i in range(len(df2)) ]
df2['house_ownership'] = [0 if df2['house_ownership'][i]=='norent_noown' else 1 if df2['house_ownership'][i]=='rented' else -1 for i in range(len(df2))]
df2['car_ownership'] = [0 if df2['car_ownership'][i]=='no' else 1 for i in range(len(df2))]



In [ ]:
num_feat = df.select_dtypes(include='number').columns.tolist()[:-1]
#cat_feat = df.select_dtypes(exclude='number').columns.tolist()
target_feat = df.select_dtypes(include='number').columns.tolist()[-1]

Y_ready = df[target_feat].values

transformer = ColumnTransformer([
                              #('encoding',OrdinalEncoder() ,cat_feat),
                              ('scaling', MinMaxScaler(), num_feat),                              
                              ], remainder='passthrough')

print(num_feat)
#print(cat_feat)
print(target_feat)



['income', 'age', 'experience', 'married', 'house_ownership', 'car_ownership', 'current_job_years', 'current_house_years']
risk_flag


In [ ]:
df = transformer.fit_transform(df.iloc[:,:-1])
df2 = transformer.transform(df2)


In [ ]:
X_ready = df
X_test = df2

X_train, X_val, y_train, y_val = train_test_split(X_ready, Y_ready, test_size=0.1,stratify=Y_ready )

print('X_train shape :', X_train.shape)
print('y_train shape :', y_train.shape)
print('\n')
print('X_val shape :', X_val.shape)
print('y_val shape :', y_val.shape)
print('\n')
print('X_test shape :', X_test.shape)



X_train shape : (226800, 8)
y_train shape : (226800,)


X_val shape : (25200, 8)
y_val shape : (25200,)


X_test shape : (28000, 8)


In [ ]:
test_data_transformed_id = pd.DataFrame(X_test)
test_data_transformed_id['id'] = test_data['id']
test_data_transformed_id

,0,1,2,3,4,5,6,7,id
0,0.739045,0.655172,0.95,0.0,1.0,0.0,0.285714,0.75,1
1,0.120594,0.068966,0.25,0.0,1.0,0.0,0.357143,0.00,2
2,0.890026,0.500000,0.60,0.0,1.0,0.0,0.642857,1.00,3
3,0.193612,0.482759,0.45,1.0,1.0,1.0,0.214286,0.50,4
4,0.000312,0.068966,0.90,0.0,1.0,1.0,0.928571,0.25,5
...,...,...,...,...,...,...,...,...,...
27995,0.995550,0.620690,0.65,0.0,1.0,0.0,0.357143,0.00,27996
27996,0.291047,0.448276,0.45,0.0,1.0,0.0,0.642857,1.00,27997
27997,0.808049,0.051724,0.25,0.0,1.0,0.0,0.285714,0.75,27998
27998,0.947370,0.517241,0.65,0.0,1.0,1.0,0.928571,1.00,27999


### RF

Rf(n_esti=300, class_weight=balanced) : Train 93.85 , Val : 93.65


Rf(n_esti=500, class_weight=balanced) : Train 93.77 , Val : 93.77

Rf(n_esti=700, class_weight=balanced) : Train 95.88 , Val : 93.77

Rf(n_esti=1000, class_weight=balanced) : Train 95.88 , Val : 93.77

In [ ]:
from sklearn.ensemble import RandomForestClassifier

for_clf = RandomForestClassifier(n_estimators=1000,
                                #  min_samples_split =50,
                                #  max_features = None,
                                 class_weight = 'balanced'
                                 
                                 )

for_clf.fit(X_train, y_train)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
#y_score_for = cross_val_predict(for_clf, X_train, y_train, cv=3, method='predict_proba')

roc_auc_score(y_train, for_clf.predict(X_train))

0.9575900937135502

In [ ]:
roc_auc_score(y_val, for_clf.predict(X_val))

0.8356342139833601

In [ ]:
y_predict_final_forest = for_clf.predict(X_test)

In [ ]:
test_data_transformed_id['risk_flag'] = y_predict_final_forest
test_data_transformed_id
Final_submission = test_data_transformed_id.iloc[:,-2:]
Final_submission


In [ ]:
Final_submission.to_csv('final_submission.csv',index=False)

In [ ]:


param_for_clf = {
                'n_estimators' : [300,400,500,600,700,800,900,1000],
                 #'min_samples_split' : int(np.random.uniform(1,20)),
                 'max_features' : ['auto', None,'sqrt'],
                 'class_weight' : ['balanced', None]
                 
    
                }

grid_for = RandomizedSearchCV(RandomForestClassifier(), param_for_clf, cv=5, verbose=4)
grid_for.fit(X_train, y_train)

In [ ]:
grid_for.best_params_

In [ ]:
print('Train Score')
y_score_for_grid = cross_val_predict(grid_for, X_train, y_train, cv=3, method='predict_proba')
roc_auc_score(y_train, y_score_for_grid[:,1])

In [ ]:
print('Val Score')
roc_auc_score(y_val, grid_for.predict_proba(X_val)[:,1])

### XGB

In [ ]:
D_train = xgb.DMatrix(X_train, label=y_train)
D_val = xgb.DMatrix(X_val, label=y_val)

xgb_clf = XGBClassifier(
                        booster = 'gbtree',
                        colsample_bytree= 0.9,
                        eta= 0.1,
                        gamma= 0.1,
                        max_depth= 14,
                        min_child_weight= 1,
                        n_estimators= 600
                        )

xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, eta=0.1, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=14,
              min_child_weight=1, missing=None, n_estimators=600, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:

y_score_xgb = cross_val_predict(xgb_clf, X_train, y_train, cv=3, method='predict_proba')

In [ ]:
roc_auc_score(y_train, y_score_xgb[:,1])

0.9243742018816887

In [ ]:
roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])

0.9251719457013575

In [ ]:
params_xgb= {
     "eta"    : [0.025,0.05, 0.10, ] ,
     'n_estimators=300' : [400,500],
     "max_depth"        : [ 8,10,12],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [  0.1, 0.2 , ],
     "colsample_bytree" : [ 0.5 , 0.7, 0.8 ],
     'booster' : ['gbtree']
     }

grid_xgb = RandomizedSearchCV(xgb_clf,params_xgb,n_iter=2, verbose=3)
grid_xgb.fit(X_train,y_train)

In [ ]:
grid_xgb.best_params_

{'booster': 'gbtree',
 'colsample_bytree': 0.8,
 'eta': 0.1,
 'gamma': 0.1,
 'max_depth': 12,
 'min_child_weight': 1,
 'n_estimators=300': 500}

In [ ]:
y_score_xgb = grid_xgb.predict_proba(X_train)
roc_auc_score(y_train, y_score_xgb[:,1])

0.9469161409780832

In [ ]:
roc_auc_score(y_val, grid_xgb.predict_proba(X_val)[:,1])

0.906212341264049

### LGBM

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(
                    nthread=4,
                    n_estimators=3000,
                    learning_rate=0.01,
                    num_leaves=5000,
                    colsample_bytree=0.9497036,
                    subsample=0.8715623,
                    max_depth=75,
                    reg_alpha=0.04,
                    reg_lambda=0.073,
                    min_split_gain=0.01,
                    min_child_weight=40,
                    silent=-1,
                    verbose=-1,
                    class_weight='balanced'
                     )

lgbm_clf.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=0.9497036, importance_type='split',
               learning_rate=0.01, max_depth=75, min_child_samples=20,
               min_child_weight=40, min_split_gain=0.01, n_estimators=3000,
               n_jobs=-1, nthread=4, num_leaves=5000, objective=None,
               random_state=None, reg_alpha=0.04, reg_lambda=0.073, silent=-1,
               subsample=0.8715623, subsample_for_bin=200000, subsample_freq=0,
               verbose=-1)

In [ ]:
print('Train score')
print(roc_auc_score(y_train, lgbm_clf.predict(X_train)))

print('Val score')
roc_auc_score(y_val, lgbm_clf.predict(X_val))

Train score
0.9314112378276316
Val score


0.8527893738140417

PLAIN LGBM

n_esti : 10000 , balanced , Train :97.53 , Val: 91.89

n_esti : 1000 , balanced , num_leaves =50 , Train :95.33 , Val: 91.67

n_esti : 1000 , balanced , num_leaves =50 , max_depth = 10 Train :95.27 , Val: 91.51


In [ ]:
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(n_estimators=1000,class_weight='balanced',num_leaves=50,)
lgbm_model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=50,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [ ]:
print('Train score')
print(roc_auc_score(y_train, lgbm_model.predict(X_train)))

print('Val score')
roc_auc_score(y_val, lgbm_model.predict(X_val))

Train score
0.9183693589537139
Val score


0.8493789227849949